<a href="https://colab.research.google.com/github/hunzlausman/ci2.2.6/blob/master/Llama_2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers[torch]
!pip install -q sentencepiece
!pip install -q langchain
!pip install -q huggingface_hub
!pip install -q llama-cpp-python==0.1.78
# !pip install -q faiss-cpu
!pip install -q faiss-gpu
!pip install -q pypdf
# !pip install -q pdfrw
!pip install -q sentence_transformers

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose
#very important for utilizing gpu resources use this only if installed cpu version of llama-cpp-python already

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.0 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-68.1.2-py3-none-any.whl (805 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.27.2-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.1 MB)
    Using cached ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (145 kB)
    Using cached distro-1.8.0-py3-none-any.whl (20 kB)
    Using cached packaging-23.1-py3-none-any.whl (48 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.41.2-py3-none-any.whl (64 kB)
    Creating /tmp/pip-build-env-5jrhmun8/overlay/local/bin
    changing mode of /tmp/pip-build-env-5jrhmun8/overlay/local/bin/ninja to 755
    changing mo

In [ ]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

In [ ]:
# !python ./convert-llama-ggmlv3-to-gguf.py --eps 1e-5 --input llama-2-7b-chat.ggmlv3.q4_K_M.bin --output llama-2-7b-chat.gguf.q4_0.bin

python3: can't open file '/content/./convert-llama-ggmlv3-to-gguf.py': [Errno 2] No such file or directory


In [ ]:
from langchain import HuggingFacePipeline
from huggingface_hub import hf_hub_download
model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_K_M.bin"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
# n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# Loading model,
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# llm.reset()
# Loading model,
# llm.reset()
# llm.set_cache(None)
llm = None
del llm
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096, # Context window
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    temperature = 0.4,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferMemory

In [ ]:
documents=PyPDFLoader("Ch2.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts = text_splitter.split_documents(documents)
# retriever.reset()
# retriever.set_cache(None)
db = FAISS.from_documents(texts,HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device':'cuda'}))
retriever = db.as_retriever()

In [ ]:
len(retriever.get_relevant_documents(""))


4

In [ ]:
from langchain import PromptTemplate, LLMChain
template = """''SYSTEM:You're a helpful assistant that can answer questions according to the following data:{content}
USER: {query}
ASSISTANT: """
prompt = PromptTemplate(template=template, input_variables=["query","content"])
memory = ConversationSummaryMemory(llm=llm,return_messages=True)

In [ ]:
template = """Given the following history summary and a follow up question,if anything in the history summary relates to the original question rewrite follow up question to be a standalone question, in its original language.

history summary:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate(template=template, input_variables=["chat_history","question"])

prompt_template = """Use the following pieces of context to answer the question at the end.
{context}

Question: {question}

Helpful Answer:"""
QA_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
question="what is my name?"
q_chain = LLMChain(prompt=CONDENSE_QUESTION_PROMPT, llm=llm)
final_query = q_chain.run(chat_history=memory.chat_memory.messages,question=question)
llm_chain = LLMChain(prompt=QA_PROMPT, llm=llm)
docs = retriever.get_relevant_documents(final_query)
print(docs)
answer = llm_chain.run(context= docs,question=final_query)
print(answer)

 What is your name?[Document(page_content='REPORT OF THE JUDICIAL FL OOD INQUIR Y TRIBUNAL, 2010   CHAPTER 2 CHAPTER 2   REPORT OF THE JUDICIAL FL OOD INQUIR Y TRIBUNAL, 2010   34 35\nSr.No. Names of public complainants. \n1.  Ch. Muhammad Yousuf, Advocate, Kot Adu (IW-24). \n2.  Mr. Muhammad Younus Chandia s/o Ghulam Rasool, r/o Mauza Bhabar \nGhair Mustaqil, Kot Adu (IW-25). \n3.  Mr. Asghar Ali Khan Pachar, Advocate, r/o Village Chaudhry, Tehsil Kot Adu \n(IW-26) \n4.  \n Syed Allah Bukhsh Shah r/o Tasneem Chah Basti Wazir Gadiwala, Mauza \nTibba Mustaqil Sharqi, Kot Adu. (IW-27) \n5.  Malik Munir Ahmad, Advocate, Kot Adu. (IW-28) \n6.  Mr. Muhammad Ashraf Khan Rind s/o Manzoor Hussain Rind, former Nazim \nUnion Council Bate Wala r/o Taunsa Barrage Colony, Tehsil Kot Adu. \n(IW-29) \n7.  \n Mr. Ghulam Abbas s/o Muhammad Bukhsh, r/o Ward No.14-C, Mohallah \nKhokhar Abad, Kot Adu (IW-31) \n8.  \n Mr. Rafique Ahmad Khan s/o Sardar Khan r/o Ward No.14-C, Kakkay Wala, \nKot Adu. (IW-32) 

Llama.generate: prefix-match hit


                                       R  C                                               R  OR  B       C    C                                             I                               
   right     R       B                         c  [  \        R 
 

 B 
  

  C                   OR  B B B B  B B B
 B  C  B B 
 i B  B R
 R B   F B  R  B  B   I  B B  O B ? --------  B  [ 
  C 
 B \  C     B  B c  F R  i    IN  OR   C  E   R  R  C  C  C  C  C  R  I  B [ and
 T
 W
 S  B B    [   [  
  
 i  [  M  B 
 
 E  A E N 
 B c[ B _%

 A F


 P C C R R R 
 IN   R  R  C    C C  C  R S
  R R                                        R  C                                               R  OR  B       C    C                                             I                               
   right     R       B                         c  [  \        R 
 

 B 
  

  C                   OR  B B B B  B B B
 B  C  B B 
 i B  B R
 R B   F B  R  B  B   I  B B  O B ? --------  B  [ 
  C 
 B \  C     B  B c  F R  i 

In [ ]:
from langchain.chains import ConversationalRetrievalChain
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

qa_chain  = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)
qa_chain(question)

In [ ]:
memory.clear()

In [ ]:
memory

In [ ]:
qa_chain("what is Pakistan")


In [ ]:
print(memory.chat_memory.messages)

In [ ]:
!sudo apt install lsb-release curl gpg

In [ ]:
!curl -fsSL https://packages.redis.io/gpg

In [ ]:
!sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

In [ ]:
!echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

In [ ]:
!sudo apt-get update
!sudo apt-get install redis

In [ ]:
!sudo apt-get install lynx

In [ ]:
!lynx http://localhost:8000

In [ ]:
!pip install chainlit

In [ ]:
import chainlit as cl
@cl.on_message
async def main(query: str):
 docs = retriever.get_relevant_documents(query)
 print((llm_chain.memory).chat_memory.messages)
 output = llm_chain.run(content=docs,query=query)
 memory.save_context({"input": query}, {"output": output})
 await cl.Message(
        content=f"Received: {message}",
    ).send()

In [ ]:
!chainlit run app.py -w

In [ ]:
!redis-cli